In [27]:
import pandas as pd
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

%matplotlib inline

In [28]:
df = pd.read_excel(r"E:\ml_projects\e2e-customer-segmentation-rfm\data\raw\online_retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [29]:
df.shape

(541909, 8)

In [30]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [31]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [32]:
df = df.dropna(subset=['CustomerID'])
print(df.shape)

(406829, 8)


In [50]:
df.TotalPrice.describe()

count    406829.000000
mean         20.401854
std         427.591718
min     -168469.600000
25%           4.200000
50%          11.100000
75%          19.500000
max      168469.600000
dtype: float64

In [33]:
df.StockCode.sample(15)

120915     22624
398189     22365
505696     71038
369670     23209
500421     23326
244485     84755
126705    16161U
442294    85049G
280423     23188
4834       82581
498568     16016
271206     23238
239506     21899
484298     23342
304645     23288
Name: StockCode, dtype: object

In [34]:
df.Country.value_counts().reset_index().head(n=10)

,Country,count
0,United Kingdom,361878
1,Germany,9495
2,France,8491
3,EIRE,7485
4,Spain,2533
5,Netherlands,2371
6,Belgium,2069
7,Switzerland,1877
8,Portugal,1480
9,Australia,1259


In [35]:
print(f"Total Customers: { len(df.CustomerID.unique())} ")

Total Customers: 4372 


In [41]:
df["TotalPrice"] = df["UnitPrice"] * df["Quantity"]
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'TotalPrice'],
      dtype='object')

In [42]:
customer_sales = df.groupby('CustomerID').agg(
    CustomerTotalSales=('TotalPrice', 'sum'),
    Country=('Country', 'first')
)

In [43]:
customer_sales

,CustomerTotalSales,Country
CustomerID,,
12346.0,0.00,United Kingdom
12347.0,4310.00,Iceland
12348.0,1797.24,Finland
12349.0,1757.55,Italy
12350.0,334.40,Norway
...,...,...
18280.0,180.60,United Kingdom
18281.0,80.82,United Kingdom
18282.0,176.60,United Kingdom


In [44]:
customer_sales = customer_sales.sort_values(by='CustomerTotalSales', ascending=False)

In [45]:
customer_sales

,CustomerTotalSales,Country
CustomerID,,
14646.0,279489.02,Netherlands
18102.0,256438.49,United Kingdom
17450.0,187482.17,United Kingdom
14911.0,132572.62,EIRE
12415.0,123725.45,Australia
...,...,...
12503.0,-1126.00,Spain
17603.0,-1165.30,United Kingdom
14213.0,-1192.20,United Kingdom


In [ ]:
total_store_sales = customer_sales['CustomerTotalSales'].sum()

# --- 4. Calculate cumulative sales and percentage ---
customer_sales['CumulativeSales'] = customer_sales['CustomerTotalSales'].cumsum()
customer_sales['CumulativePercentage'] = customer_sales['CumulativeSales'] / total_store_sales

# --- 5. Filter to find the customers in the top 10% ---
top_10_percent_customers = customer_sales[customer_sales['CumulativePercentage'] <= 0.10]

# --- 6. Display the result ---
print(f"Total Store Sales: ${total_store_sales:,.2f}")


Total Store Sales: $8,300,065.81
--- Top Customers Contributing to 10% of Sales ---


In [47]:
print("--- Top Customers Contributing to 10% of Sales ---")
top_10_percent_customers_display = top_10_percent_customers[[
    'Country', 
    'CustomerTotalSales', 
    'CumulativePercentage'
]].copy()

# Format the numbers for readability
top_10_percent_customers_display['CustomerTotalSales'] = top_10_percent_customers_display['CustomerTotalSales'].map('${:,.2f}'.format)
top_10_percent_customers_display['CumulativePercentage'] = top_10_percent_customers_display['CumulativePercentage'].map('{:.2%}'.format)

print(top_10_percent_customers_display)

print(f"\n{len(top_10_percent_customers)} customers account for the first 10% of all sales.")

--- Top Customers Contributing to 10% of Sales ---
                   Country CustomerTotalSales CumulativePercentage
CustomerID                                                        
14646.0        Netherlands        $279,489.02                3.37%
18102.0     United Kingdom        $256,438.49                6.46%
17450.0     United Kingdom        $187,482.17                8.72%

3 customers account for the first 10% of all sales.


In [54]:
df[df['TotalPrice'] == 168469.60]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,168469.6


In [55]:
df[df['TotalPrice'] == -168469.60]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,-168469.6


In [59]:
df[df['InvoiceNo'].str.startswith('C').fillna(False)]

C:\Users\Future\AppData\Local\Temp\ipykernel_12644\3489678364.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[df['InvoiceNo'].str.startswith('C').fillna(False)]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,-27.50
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,-4.65
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,-19.80
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96
...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom,-9.13
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,-224.69
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,-54.75
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,-1.25


In [60]:
# Create a DataFrame of positive purchases
purchases_df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Create a DataFrame of negative returns/cancellations
returns_df = df[df['Quantity'] < 0]

In [ ]:
# --- Get Purchase-related features ---
customer_purchases = purchases_df.groupby('CustomerID').agg(
    TotalPurchaseTotalPrice=('TotalPrice', 'sum'),
    PurchaseFrequency=('InvoiceNo', 'nunique')
)

# --- Get Return-related features ---
customer_returns = returns_df.groupby('CustomerID').agg(
    TotalReturnTotalPrice =('TotalPrice', 'sum'), # This will be negative
    ReturnFrequency=('InvoiceNo', 'nunique')
)

# --- Combine them into one customer-level DataFrame ---
customer_full_profile = customer_purchases.join(customer_returns, how='left')

# Fill NaN with 0 for customers who have 0 returns
customer_full_profile = customer_full_profile.fillna(0)

In [ ]:
# We use abs() to make the return TotalPrice  positive for a simple ratio
customer_full_profile['ReturnRate'] = (
    customer_full_profile['TotalReturnTotalPrice'].abs() / 
    customer_full_profile['TotalPurchaseTotalPrice']
)

# Handle cases where PurchaseTotalPrice is 0 (if any)
customer_full_profile['ReturnRate'] = customer_full_profile['ReturnRate'].fillna(0)

In [63]:
customer_full_profile

,TotalPurchaseAmount,PurchaseFrequency,TotalReturnAmount,ReturnFrequency,ReturnRate
CustomerID,,,,,
12346.0,77183.60,1,-77183.60,1.0,1.000000
12347.0,4310.00,7,0.00,0.0,0.000000
12348.0,1797.24,4,0.00,0.0,0.000000
12349.0,1757.55,1,0.00,0.0,0.000000
12350.0,334.40,1,0.00,0.0,0.000000
...,...,...,...,...,...
18280.0,180.60,1,0.00,0.0,0.000000
18281.0,80.82,1,0.00,0.0,0.000000
18282.0,178.05,2,-1.45,1.0,0.008144


In [64]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'TotalPrice'],
      dtype='object')

In [ ]:
df_cleaned = df[df['Quantity'] > 0]
df_cleaned = df_cleaned[df_cleaned['UnitPrice'] > 0]

In [67]:
df_cleaned['TotalPrice'].describe()

count    397884.000000
mean         22.397000
std         309.071041
min           0.001000
25%           4.680000
50%          11.800000
75%          19.800000
max      168469.600000
Name: TotalPrice, dtype: float64

In [68]:
df.InvoiceDate.max()

Timestamp('2011-12-09 12:50:00')

In [69]:
df.InvoiceDate.min()

Timestamp('2010-12-01 08:26:00')

In [72]:
import datetime

# 1. Convert to datetime
df['invdatetime'] = pd.to_datetime(df.InvoiceDate)

# 2. Get reference date from the *datetime* column
reference_date = df['invdatetime'].max() + datetime.timedelta(days = 1)

# 3. Calculate the timedelta using the *datetime* column
df['days_since_last_purchase'] = reference_date - df['invdatetime']

# 4. Extract the days using .dt.days (THE FIX)
df['days_since_last_purchase_num'] = df['days_since_last_purchase'].dt.days

# Check the result
print(df[['invdatetime', 'days_since_last_purchase', 'days_since_last_purchase_num']].head())

          invdatetime days_since_last_purchase  days_since_last_purchase_num
0 2010-12-01 08:26:00        374 days 04:24:00                           374
1 2010-12-01 08:26:00        374 days 04:24:00                           374
2 2010-12-01 08:26:00        374 days 04:24:00                           374
3 2010-12-01 08:26:00        374 days 04:24:00                           374
4 2010-12-01 08:26:00        374 days 04:24:00                           374


In [74]:
customer_history_df = df.groupby("CustomerID")[['days_since_last_purchase_num']].min().reset_index()
customer_history_df.rename(columns={'days_since_last_purchase_num': 'recency'}, inplace=True)
customer_history_df.recency.describe()

count    4372.000000
mean       92.047118
std       100.765435
min         1.000000
25%        17.000000
50%        50.000000
75%       143.000000
max       374.000000
Name: recency, dtype: float64

In [75]:
customer_history_df

,CustomerID,recency
0,12346.0,326
1,12347.0,2
2,12348.0,75
3,12349.0,19
4,12350.0,310
...,...,...
4367,18280.0,278
4368,18281.0,181
4369,18282.0,8
4370,18283.0,4


In [77]:
customer_monetary_val = df[['CustomerID', 'TotalPrice']].groupby("CustomerID").sum().reset_index()
customer_history_df = customer_history_df.merge(customer_monetary_val, how='outer')
customer_history_df.TotalPrice = customer_history_df.TotalPrice+0.001
customer_freq = df[['CustomerID', 'TotalPrice']].groupby("CustomerID").count().reset_index()
customer_freq.rename(columns={'TotalPrice':'frequency'},inplace=True)
customer_history_df = customer_history_df.merge(customer_freq, how='outer')

In [80]:
customer_history_df.rename(columns={"TotalPrice": "monetary"}, inplace=True)

In [82]:
customer_history_df = customer_history_df[['CustomerID', 'recency', 'frequency', 'monetary']]
customer_history_df

,CustomerID,recency,frequency,monetary
0,12346.0,326,2,0.001
1,12347.0,2,182,4310.001
2,12348.0,75,31,1797.241
3,12349.0,19,73,1757.551
4,12350.0,310,17,334.401
...,...,...,...,...
4367,18280.0,278,10,180.601
4368,18281.0,181,7,80.821
4369,18282.0,8,13,176.601
4370,18283.0,4,756,2094.881


In [83]:
# Define the full file path
file_path = r"E:\ml_projects\e2e-customer-segmentation-rfm\data\interim\customer_history.csv"

# Save the DataFrame
customer_history_df.to_csv(file_path, index=False)